In [1]:
import sys
from multi_blade import MultiBlade

In [2]:
import copy
import logging
import pickle
from pathlib import Path
import time
from termcolor import colored
import pandas as pd
import numpy as np
from doc_selector import DocSelector
from scipy.sparse import csr_matrix, hstack, vstack
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsRestClassifier
from termcolor import colored
from IPython.display import display, HTML, clear_output

In [3]:
blade = MultiBlade(
   model_path = "/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/models/filtering/rosie_1_20",
    source_path="/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/source/corpus_rosie/passages/translated/df_1.parquet",
    lang = "ES",
    blade_state_path='/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/models/filtering/rosie_1_20/multi_blade_state_ES_trained.pkl'
)

In [4]:
df = pd.read_csv("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/models/filtering/rosie_1_20/df_docs_updated_ES.csv")
df[df.human_labeled==True]

/tmp/ipykernel_1008353/1555556115.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/models/filtering/rosie_1_20/df_docs_updated_ES.csv")


,lemmas_x_x,id_top,doc_id,id_preproc,lemmas_x_y,text,len,label,human_labeled
1,especial caso meduloblastomas desenlace relaci...,1,ES_490955_64059-16,1,especial caso meduloblastomas desenlace relaci...,"Cada vez está más claro, en especial en el cas...",36,"[1, 1, 0, 0]",True
22403,proveedor_atención médica recomendar él leche_...,22403,ES_721057_99151-14,22403,proveedor_atención médica recomendar u él lech...,Si el proveedor de atención médica recomienda ...,50,"[0, 1, 1, 0]",True
41870,consumo_tabaco padre dañar salud niño familia ...,41870,ES_515156_64369-6,41870,consumo_tabaco padre dañar salud niño familia ...,“El consumo de tabaco por los padres daña la s...,22,"[1, 1, 0, 0]",True
48233,zobel hermano vivir tío familia long_island in...,48233,ES_816514_104975-3,48233,zobel hermano vivir tío familia long_island in...,Zobel y su hermano vivían con su tío y su fami...,67,"[0, 0, 1, 0]",True
51523,herramienta_validada,51523,ES_670984_7426-75,51523,herramienta_validada,"Además, sin necesidad de emplear para ello her...",1,"[0, 0, 0, 0]",True
...,...,...,...,...,...,...,...,...,...
1053335,tipo prueba genético tipo prueba genético,1053364,ES_298222_40937-9,1053364,tipo prueba genético tipo prueba genético,¿Cuáles son los tipos principales de pruebas g...,6,"[1, 0, 0, 0]",True
1053338,diagnosticar afta_afta frecuente significar ig...,1053367,ES_349351_42861-13,1053367,diagnosticar afta_afta frecuente significar ig...,¿Cómo se diagnostican las aftas: Las aftas son...,15,"[1, 0, 0, 0]",True
1053389,síntoma grave síndrome vejiga doloroso afectar...,1053418,ES_617082_86765-10,1053418,síntoma grave síndrome vejiga doloroso afectar...,Estas son algunas maneras en que los síntomas ...,46,"[1, 0, 1, 0]",True
1053401,ultrasonido cráneo estudio producir dolor onda...,1053430,ES_726143_99370-9,1053430,ultrasonido cráneo estudio producir dolor onda...,\n- Ultrasonido del cráneo. Estudio que no pro...,33,"[1, 1, 0, 0]",True


In [5]:
filtered_df = df[df['label'].apply(lambda x: x == '[0, 0, 0, 0]')]
filtered_df

,lemmas_x_x,id_top,doc_id,id_preproc,lemmas_x_y,text,len,label,human_labeled
51523,herramienta_validada,51523,ES_670984_7426-75,51523,herramienta_validada,"Además, sin necesidad de emplear para ello her...",1,"[0, 0, 0, 0]",True
177820,inclinar_sentimiento idea posición decisión_in...,177824,ES_233370_39176-40,177824,inclinar_sentimiento idea posición decisión_in...,¿Hacia qué se inclina ahora: Ahora que ha cons...,5,"[0, 0, 0, 0]",True
479741,matriculado acceder participar ecoe_abandonar ...,479755,ES_992415_8527-26,479755,matriculado acceder participar ecoe_abandonar ...,"De los 113 matriculados, 102 accedieron a part...",13,"[0, 0, 0, 0]",True
730227,hoja electrónico calcular_salario gastado_dine...,730246,ES_94997_22083-0,730246,use hoja electrónico calcular_salario gastado_...,Use esta hoja de trabajo electrónica para calc...,13,"[0, 0, 0, 0]",True
812804,reconocer importancia hombre mujer,812827,ES_31805_10992-390,812827,reconocer importancia hombre mujer,• Reconocer la importancia de las diferencias ...,4,"[0, 0, 0, 0]",True
925470,obvio gluten alimentos panes rosca pan croissa...,925498,ES_936862_123473-5,925498,obvio gluten alimentos panes rosca pan croissa...,Las fuentes obvias de gluten incluyen:\n- Alim...,23,"[0, 0, 0, 0]",True
948370,género manfreda pertenecer familia asparagacea...,948398,ES_1057085_9955-2,948398,género manfreda pertenecer familia asparagacea...,El género Manfreda pertenece a la familia Aspa...,26,"[0, 0, 0, 0]",True
1037943,discriminación sesgo atención médico home you_...,1037972,ES_38127_11391-0,1037972,discriminación sesgo atención médico home you_...,Discriminación y sesgo en la atención médica H...,20,"[0, 0, 0, 0]",True


In [6]:
pred = blade.predict(path_save = "/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/models/filtering/rosie_1_20/df_docs_predicted_ES_multi.csv")

NameError: name 'f' is not defined

In [ ]:
#blade.active_learning_loop(max_duration_minutes=1)
#blade.get_document(0)
#blade.do_update([1,0,1,0])